# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [52]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

In [53]:
df = pd.read_csv('/datasets/data.csv')

In [54]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


 Итак, первым делом прочитали файл '/datasets/data.csv' и сохранили его в переменной df. Затем получили общую информацию о данных в таблице. Всего мы видим 12 столбцов. Подробно рассмотрим названия столбцов и какой смысл они за собой несут:
 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

In [55]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Методом describe получили общую информацию о данных в таблице. Сразу бросилось в глаза несколько моментов:
1) Минимальное значение по количеству детей 'children' = (-1), пока не понятно что это за значение, вполне возможно что ошибка выгрузки данных, так же по этому столбцу максимальное значение принимает 20. Это более реалистично, но нужна проверка
2) Отрицательные значения в столбце с трудовым стажем 'days_employed'. Возможно значения взяты с разных ресурсов или опять же виновата некорректная выгрузка данных. Вполне возможно что при взятии модуля - данные подойдут для обработки. Так же бросилось в глаза что среднее значение по этому столбцу составляет 63046 дней, а это 172 года, скорее всего где-то есть ошибка
3) Возраст некоторых клиентов почему-то 0.(Ясно, заемщику 0 лет)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Отсутствие данных в столбцах 'days_employed' и 'total_income' говорит нам о том что в них присутствуют пустые значения 

In [57]:
df[(df['days_employed'].isnull() == True) & (df['total_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Теперь мы понимаем, что если значения отсутствуют по 'days_employed', то отсутствуют и по 'total_income'. Может это связано с пенсионерами?

In [58]:
print("Количество строк с 'days_employed' > 0:", df[df['days_employed'] > 0].shape[0])
print("Количество строк с 'days_employed' < 0:", df[df['days_employed'] < 0].shape[0])

Количество строк с 'days_employed' > 0: 3445
Количество строк с 'days_employed' < 0: 15906


Посмотрим на пенсионеров:

In [59]:
df_pensioner = df[(df['days_employed'] > 0) & (df['income_type'] == 'пенсионер')]
print("Количество пенсионеров с 'days_employed' > 0:", df_pensioner.shape[0])
print("Среднее количество пенсионеров с 'days_employed' > 0:", df_pensioner['days_employed'].mean())
print("Количество пенсионеров с 'days_employed' > 0 и трудовым стажем больше 20 лет:", df[df['days_employed'] > 7300].shape[0])

Количество пенсионеров с 'days_employed' > 0: 3443
Среднее количество пенсионеров с 'days_employed' > 0: 365003.4912448612
Количество пенсионеров с 'days_employed' > 0 и трудовым стажем больше 20 лет: 3445


Большинство значений столбца 'days_employed' - отрицательные, к пенсионерам они отношения не имеют. Значит можно просто взять весь столбец по модулю - так мы исправим отрицательные значения.  
Теперь "пробежимся" по детям и людям, чей возраст равен 0:

In [60]:
print("Количество людей, чей возраст =0:", df[df['dob_years'] == 0].count()[0])
print("Количество людей с -1 ребенком:", df[df['children'] == -1].count()[0])
print("Количество людей с 20 детьми:", df[df['children'] == 20].count()[0])

Количество людей, чей возраст =0: 101
Количество людей с -1 ребенком: 47
Количество людей с 20 детьми: 76


Скорее всего там, где детей -1 - просто ошибка. Будем брать по модулю. Так же мне не нравится количество людей с 20 детьми. Выглядит очень не правдоподобно, я думаю что ноль после двойки - лишний. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Итак, для того чтобы уйти от отрицательных значений в days_employed и total_income - мы возьмём их значения по модулю:

In [61]:
df[['total_income', 'days_employed']] = df[['total_income', 'days_employed']].abs()
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Заменим NaN в 'days_employed' и 'income_type':

In [62]:
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median()

In [63]:
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median()

In [64]:
df['education'] = df['education'].str.lower()
#для начала приведу весь столбец с образованием в нижний регистр чтобы убрать повторяющиеся значения

In [65]:
medians = (df.groupby(['family_status', 'education']).agg({'days_employed': 'median'})\
.rename(columns = {'days_employed': 'median_days_employed'}))
df = df.merge(medians, on = ['family_status','education'])
print(medians)

                                           median_days_employed
family_status         education                                
Не женат / не замужем высшее                        1816.871007
                      начальное                     1205.185626
                      неоконченное высшее            910.394890
                      среднее                       1856.018903
                      ученая степень                 409.200149
в разводе             высшее                        2194.220567
                      начальное                     1124.359053
                      неоконченное высшее           1355.477051
                      среднее                       2219.855114
                      ученая степень                2351.431934
вдовец / вдова        высшее                        6059.989208
                      начальное                   349351.643438
                      неоконченное высшее           8244.304591
                      среднее           

In [66]:
df[['family_status','education','days_employed','median_days_employed']]\
[df['days_employed'] > 0].tail(20)

,family_status,education,days_employed,median_days_employed
21505,в разводе,начальное,2706.976924,1124.359053
21506,вдовец / вдова,неоконченное высшее,681.343142,8244.304591
21507,вдовец / вдова,неоконченное высшее,8244.304591,8244.304591
21508,вдовец / вдова,неоконченное высшее,1554.063508,8244.304591
21509,вдовец / вдова,неоконченное высшее,395662.090486,8244.304591
21510,вдовец / вдова,неоконченное высшее,337286.878595,8244.304591
21511,вдовец / вдова,неоконченное высшее,657.416731,8244.304591
21512,вдовец / вдова,неоконченное высшее,672.209633,8244.304591
21513,вдовец / вдова,неоконченное высшее,2339.161355,8244.304591
21514,вдовец / вдова,неоконченное высшее,395524.382231,8244.304591


Почему tail а не head? Мне кажется что по tail видно гораздо лучше, т.к. head мне практически ничего не показал

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Супер! Хочу показать тебе лаконичный вариант замены:
    
    
    df['total_income'] = df.groupby(['income_type','family_status', 'education'])['total_income'].apply(lambda x: x.fillna(x.median()))

    
Оно же, но без лямбды:
    
    
    df['total_income'] = df['total_income'].fillna(df.groupby(['income_type','family_status', 'education'])['total_income'].transform('median'))


  
Необязательно выводить через head или tail. Можно вывести n случайных строк:    
</div>

In [67]:
# КОД РЕВЬЮЕРА
df.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed
16308,0,366963.310922,69,среднее,1,вдовец / вдова,2,F,пенсионер,0,100293.874813,покупка жилой недвижимости,329294.471845
5337,0,573.706292,36,среднее,1,женат / замужем,0,F,сотрудник,0,179829.570793,сделка с автомобилем,2194.220567
20231,0,1293.754476,31,высшее,0,Не женат / не замужем,4,M,госслужащий,0,92812.443657,жилье,1816.871007
14080,1,188.732581,50,среднее,1,гражданский брак,1,F,компаньон,0,76090.278173,сыграть свадьбу,2194.220567
16274,0,217.021314,36,среднее,1,вдовец / вдова,2,F,сотрудник,0,66165.148572,ремонт жилью,329294.471845
20622,0,539.778603,22,неоконченное высшее,2,Не женат / не замужем,4,M,госслужащий,0,114050.811253,покупка жилой недвижимости,910.394890
19627,1,2146.950218,36,начальное,3,женат / замужем,0,F,компаньон,0,273306.180534,покупка жилья для сдачи,2265.962208
4780,0,3063.204024,36,среднее,1,женат / замужем,0,F,сотрудник,0,110043.750079,образование,2194.220567
2809,0,2030.841108,27,высшее,0,женат / замужем,0,F,сотрудник,1,158486.333214,свой автомобиль,2194.220567
11716,0,2191.211785,41,среднее,1,женат / замужем,0,F,сотрудник,0,216797.904760,покупка коммерческой недвижимости,2194.220567


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children                21525 non-null int64
days_employed           21525 non-null float64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null float64
purpose                 21525 non-null object
median_days_employed    21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.3+ MB


Как итог - убрали NaN из трудового стажа и общего дохода

### Замена типа данных

По некоторым значениям прослеживаются некоторые "артефакты", например трудовой стаж в 'income_type' = 'пенсионер' > 90 лет!!;
В столбце 'education' явно есть неоднотипные значения, например ВЫСШЕЕ и высшее или Среднее и среднее, необходимо привести к одному значению;
Так же исправим минимальное значение по 'children' с (-1) на 0

In [69]:
#привели всё в нижний регистр
df['education'] = df['education'].str.lower()

In [70]:
print("Количество детей с (-1):", df[df['children'] == -1].count()[0])
print("Количество детей с 0:", df[df['children'] == 0].count()[0])
print("Количество детей с 20:", df[df['children'] == 20].count()[0])

Количество детей с (-1): 47
Количество детей с 0: 14149
Количество детей с 20: 76


Мы видим что заемщиков с количеством детей -1 всего 47, по отношению с теми у кого 0 - ничтожно мало. Я думаю что можно изменить с -1 на 0, статистике не помешает, но смотреться будет лучше
Тех у кого 20 детей, заменим на 2, будет выглядеть правдоподобнее

In [71]:
df['children'] = df['children'].replace(-1, 0)

In [72]:
#проверим по столбцу dob_years количество "нулевого" возраста и несовершеннолетних
print("Количество строк с 'dob_years' = 0 -", df[df['dob_years'] == 0].count()[0])
print("Количество строк с 'dob_years' <0 -", df[df['dob_years'] < 19].count()[0])

Количество строк с 'dob_years' = 0 - 101
Количество строк с 'dob_years' <0 - 101


Количество строк совпадает. Могу предположить что это одни и те же люди. Заполним по ним средние значения по каждой группе

In [73]:
df['dob_years'] = df.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))
print("Количество строк с 'dob_years' = 0 -", df[df['dob_years'] == 0].count()[0])

Количество строк с 'dob_years' = 0 - 0


In [74]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,median_days_employed
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04,21525.000000
mean,0.541092,60378.032733,43.495238,0.817236,0.972544,0.080883,1.651595e+05,14536.493967
std,1.379943,133257.558514,12.230322,0.548138,1.420324,0.272661,9.786607e+04,62482.619155
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04,409.200149
25%,0.000000,1025.608174,34.000000,1.000000,0.000000,0.000000,1.077982e+05,2194.220567
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,1.450179e+05,2194.220567
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05,2194.220567
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06,349351.643438


Переведем 'days_employed' и 'total_income' в int. Таким образом получим более красивую картинку

In [75]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.dtypes

children                  int64
days_employed             int64
dob_years                 int64
education                object
education_id              int64
family_status            object
family_status_id          int64
gender                   object
income_type              object
debt                      int64
total_income              int64
purpose                  object
median_days_employed    float64
dtype: object

In [76]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,2194.220567
1,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,2194.220567
2,2,4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,покупка недвижимости,2194.220567
3,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,2194.220567
4,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,строительство собственной недвижимости,2194.220567
5,0,335,36,высшее,0,женат / замужем,0,M,сотрудник,0,414404,недвижимость,2194.220567
6,0,2194,52,высшее,0,женат / замужем,0,F,пенсионер,0,145017,покупка жилья для семьи,2194.220567
7,2,2152,46,высшее,0,женат / замужем,0,M,компаньон,0,592071,операции с коммерческой недвижимостью,2194.220567
8,1,2802,28,высшее,0,женат / замужем,0,M,госслужащий,0,207561,покупка коммерческой недвижимости,2194.220567
9,1,2194,32,высшее,0,женат / замужем,0,M,госслужащий,0,145017,операции с коммерческой недвижимостью,2194.220567


### Обработка дубликатов

Посмотрим, сколько же у нас дубликатов:

In [77]:
df.duplicated().sum()

71

Прежде чем удалять их, посмотрим на них:

In [78]:
df[df.duplicated(keep = False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed
87,1,2194,30,высшее,0,женат / замужем,0,F,госслужащий,0,145017,покупка жилья для семьи,2194.220567
312,0,2194,54,высшее,0,женат / замужем,0,M,компаньон,0,145017,операции с коммерческой недвижимостью,2194.220567
469,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости,2194.220567
1208,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости,2194.220567
1259,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,145017,получение образования,2194.220567
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19018,0,2194,50,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,недвижимость,1856.018903
19107,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем,1856.018903
19400,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем,1856.018903
19778,0,2194,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование,1816.871007


Видим что совпало несколько значений, но мы не видим все дубликаты, попробуем их убрать:

In [79]:
df = df.drop_duplicates()

In [80]:
#Проверим количество дубликатов:
df.duplicated().sum()

0

### Лемматизация

Воспользуемся модулем pymystem3 и произведем лемматизацию по столбцу purpose. Выведем сразу уникальные значения:

In [81]:
from pymystem3 import Mystem
m = Mystem()
print(df['purpose'].unique())
purpose_credit="""
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
 """

['покупка жилья' 'операции с жильем' 'покупка недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'покупка жилья для семьи' 'операции с коммерческой недвижимостью'
 'покупка коммерческой недвижимости' 'на покупку подержанного автомобиля'
 'сделка с автомобилем' 'жилье' 'автомобиль' 'профильное образование'
 'покупка жилья для сдачи' 'операции со своей недвижимостью' 'автомобили'
 'приобретение автомобиля' 'на покупку своего автомобиля'
 'покупка своего жилья' 'покупка жилой недвижимости'
 'заняться высшим образованием' 'получение высшего образования'
 'сделка с подержанным автомобилем' 'ремонт жилью' 'свой автомобиль'
 'строительство жилой недвижимости' 'высшее образование'
 'строительство недвижимости' 'операции с недвижимостью'
 'получение образования' 'образование' 'заняться образованием'
 'на покупку автомобиля' 'дополнительное образование'
 'получение дополнительного образования' 'сыграть свадьбу'
 'на проведение свадьбы' 'свадьба']


Получив названия всех встречающихся целей на кредит, посмотрим сколько из них встречается чаще всего:

In [48]:
lemmas = m.lemmatize(purpose_credit)
from collections import Counter
print(Counter(lemmas))

Counter({' ': 59, "' '": 23, "'\n": 14, " '": 14, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, '\n': 1, "['": 1, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, "']\n": 1, ' \n': 1})


А теперь уберём всё лишнее и оставим топ 4 категорий:

In [49]:
def find_cat(row_values):
    lemm = m.lemmatize(row_values['purpose'])
    if 'жилье' in lemm: 
        return 'на недвижимость'
    if 'недвижимость' in lemm:
        return 'на недвижимость'
    if 'автомобиль' in lemm:
        return 'на автомобиль'
    if 'операция' in lemm:
        return 'на операция'
    if 'свадьба' in lemm:
        return 'на свадьба'
    if 'образование' in lemm:
        return 'на образование'
 
df['purpose'] = df.apply(find_cat,axis=1)
print(df['purpose'].value_counts())

на недвижимость    10811
на автомобиль       4306
на образование      4013
на свадьба          2324
Name: purpose, dtype: int64


<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Согласна с категориями. А копипаст зачем? Потому что вылезает ошибка о том, что нельзя к списку применять строковый метод? </div>

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера 2</h2>
    
Полагаю, что да. Можно сразу в цикле все делать, чтобы не копировать этот список строк.  </div>

```python

lemmas = []

for purpose in df['purpose'].unique():
    lemmas.append(''.join(m.lemmatize(purpose)).strip())
 
from collections import Counter
Counter(lemmas)    
```

Как итог - получили количество строк по самым популярным значениям

### Категоризация данных

Проверим корректность всех пар 'education'-'education_id' и 'family_status'-'family_status_id'

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Да, это те самые словари, о которых нас спрашивают. </div>

In [32]:
df.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


In [33]:
df.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2810
1,в разводе,3,1195
2,вдовец / вдова,2,959
3,гражданский брак,1,4151
4,женат / замужем,0,12339


Все пары уникальны. Теперь разобьем классы по уровню дохода ('salary')  на бедных, среднедоходных, богатых; по рабочему стажу ('days_employed') на < 10лет, 10-30 лет, > 30 лет; по возрасту ('dob_years') < 30 лет, 30 - 60 лет, > 60 лет; по количеству детей ('children') на 0 детей, 1-2 ребенка, > 3 детей

In [34]:
def total_income_cat(row):
    if row['total_income'] <= 30000:
        return 'бедный'
    elif 30000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] <= 500000:
        return 'выше среднего'
    else:
        return 'богатый'

    
def days_employed_cat(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

    
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [35]:
df['total_income_cat'] = df.apply(total_income_cat, axis=1)
df['days_employed_cat'] = df.apply(days_employed_cat, axis=1)
df['dob_years_cat'] = df.apply(dob_years_cat, axis=1)
df['children_cat'] = df.apply(children_cat, axis=1)

In [36]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_days_employed,total_income_cat,days_employed_cat,dob_years_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,на недвижимость,2194.220567,выше среднего,стаж более 30 лет,30-45 лет,1-2 ребенка
1,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,30-45 лет,нет детей
2,2,4171,36,высшее,0,женат / замужем,0,M,компаньон,0,113943,на недвижимость,2194.220567,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
3,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,до 30 лет,нет детей
4,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,до 30 лет,1-2 ребенка
5,0,335,36,высшее,0,женат / замужем,0,M,сотрудник,0,414404,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,30-45 лет,нет детей
6,0,2194,52,высшее,0,женат / замужем,0,F,пенсионер,0,145017,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,45-65 лет,нет детей
7,2,2152,46,высшее,0,женат / замужем,0,M,компаньон,0,592071,на недвижимость,2194.220567,богатый,стаж до 10 лет,45-65 лет,1-2 ребенка
8,1,2802,28,высшее,0,женат / замужем,0,M,госслужащий,0,207561,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,до 30 лет,1-2 ребенка
9,1,2194,32,высшее,0,женат / замужем,0,M,госслужащий,0,145017,на недвижимость,2194.220567,выше среднего,стаж до 10 лет,30-45 лет,1-2 ребенка


Мы категоризировали нашу таблицу по нескольким параметрам: 
-по уровню зарплаты ('salary');
-по стажу работы ('days_employed');
-по количеству детей ('children').

## Шаг 3. Ответьте на вопросы

In [37]:
def relation(category):
    return df.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
relation('children_cat')

,debt
children_cat,
нет детей,0.075258
многодетные,0.085526
1-2 ребенка,0.093003


<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Можно еще процент приписать: </div>

In [39]:
# КОД РЕВЬЮЕРА
df.groupby('children_cat').agg({'debt':lambda x: str(round(x.mean()*100,2)) +'%' })

,debt
children_cat,
1-2 ребенка,9.3%
многодетные,8.55%
нет детей,7.53%


Вывод

Заёмщики не имеющие детей реже всего попадают в просрочки

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
relation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


In [41]:
df.groupby('family_status').agg({'debt':lambda x: str(round(x.mean()*100,2)) +'%' })

,debt
family_status,
Не женат / не замужем,9.75%
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.35%
женат / замужем,7.55%


**Вывод**

Более всего склонны к просрочкам холостяки и те кто живут в гражданском браке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
relation('total_income_cat')

,debt
total_income_cat,
богатый,0.063063
средний,0.080463
выше среднего,0.081740
бедный,0.090909


In [43]:
df.groupby('total_income_cat').agg({'debt':lambda x: str(round(x.mean()*100,2)) +'%' })

,debt
total_income_cat,
бедный,9.09%
богатый,6.31%
выше среднего,8.17%
средний,8.05%


**Вывод**

Очень интересно, но самыми ответственными по возврату кредитов в срок являются люди с доходом  меньше 50 т.р.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Логично. Как правило, люди с маленьким доходом имеют совсем иные приоритеты, в отличие от тех, кто много себе позволяет. Они просто не потянут комиссии за просрочки, поэтому стараются платить вовремя. </div>

- Как разные цели кредита влияют на его возврат в срок?

In [44]:
relation('purpose')

,debt
purpose,
на недвижимость,0.072334
на свадьба,0.080034
на образование,0.092200
на автомобиль,0.093590


In [45]:
df.groupby('purpose').agg({'debt':lambda x: str(round(x.mean()*100,2)) +'%' })

,debt
purpose,
на автомобиль,9.36%
на недвижимость,7.23%
на образование,9.22%
на свадьба,8.0%


**Вывод**

Самые ответственные - те кто платит за жильё

## Шаг 4. Общий вывод

Подведём итоги и ответим на главный интересующий нас вопрос - Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок? - Да, влияет. Как показало наше исследование - самые надежные заемщики это те, которые не имеют детей, холосты, чей уровень дохода меньше 50 тысяч и целью кредита является жильё

<div class="alert alert-warning" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid">
<b> Комментарий ревьюера</b>
    
Хорошо. В идеале надо бы привести проценты невозврата в срок у самых надежных и самых безответственных клиентов для каждой группы. Например: самые надежные клиенты это люди, берущие кредит на операции с недвижимостью (такой-то процент невозврата в срок), а самые безответственные те, кто обращается в банк с целью получить деньги на покупку автомобиля (такой-то процент). И так для каждый группы. Это хороший тон, — подкреплять вывод полученными ранее результатами. Кроме того, так ты не оставляешь заказчику шансов неверно интерпретировать вывод.
</div>

<div class="alert alert-info">
<h2> Комментарий студента </h2>
    Спасибо! К каждой категории проставил вывод в процентах
   

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.